In [58]:
from bs4 import BeautifulSoup

import pandas as pd
import pickle
import requests

In [ ]:
with open("morningstar_etf_data.pkl", "rb") as f:
    etf_data = pickle.load(f)

In [40]:
# Go to https://www.morningstar.com/5-star-etfs and see how many pages there are on the site. Set num_pages
# accordingly and the url accordingly and run. Repeat for all star etfs down to 1 star.

num_pages = 2
for i in range(1, num_pages+1):
    url = "https://www.morningstar.com/1-star-etfs?page={}".format(i)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
    }
    response = requests.get(url, headers = headers)

    soup = BeautifulSoup(response.content, 'html.parser')

    rows = soup.find_all(class_="mdc-table-row")

    for row in rows:
        data_points = row.find_all(class_="mdc-data-point")
        data_points_list = []
        for data_point in data_points:
            data_points_list.append(data_point.text)
        if len(data_points_list) != 8:
            continue
        name, ticker, morningstar_category, adjusted_expense_ratio, return_rank_in_category_1Y, return_rank_in_category_3Y, return_rank_in_category_5Y, active_or_passive = data_points_list
        etf_data[ticker] = {
            'name': name,
            'morningstar_category': morningstar_category,
            'adjusted_expense_ratio': adjusted_expense_ratio,
            'return_rank_in_category_1Y': return_rank_in_category_1Y,
            'return_rank_in_category_3Y': return_rank_in_category_3Y,
            'return_rank_in_category_5Y': return_rank_in_category_5Y,
            'active_or_passive': active_or_passive
        }

In [56]:
with open("morningstar_etf_data.pkl", "wb") as f:
    pickle.dump(etf_data, f)

In [ ]:
for ticker in etf_data:
    url = "https://query1.finance.yahoo.com/v7/finance/download/{}?period1=1370044800&period2=1685577600&interval=1d&events=history&includeAdjustedClose=true".format(ticker)
    response = requests.get(url, headers = headers)

    save_path = "Desktop/etf_historical_prices/{}.csv".format(ticker)
    with open(save_path, 'wb') as file:
        file.write(response.content)
    
    print(ticker)

In [83]:
points_per_etf = {}

spy = pd.read_csv("Desktop/etf_historical_prices/SPY.csv")
spy_movements = []
for i in range(len(spy['Close']) - 1, 0, -1):
    spy_movements.append(100*(spy['Close'][i]/spy['Close'][i-1] - 1))

In [ ]:
for ticker in etf_data:
    other = pd.read_csv("Desktop/etf_historical_prices/{}.csv".format(ticker))
    other_movements = []
    for i in range(len(other['Close']) - 1, 0, -1):
        other_movements.append(100*(other['Close'][i]/other['Close'][i-1] - 1))
        
    points = [0, 0, 0, 0]
    
    for i in range(len(other_movements)):
        if spy_movements[i] > 0:
            points[0] += other_movements[i] - spy_movements[i]
        else:
            points[1] += (other_movements[i] - spy_movements[i])*(abs(other_movements[i] - spy_movements[i]))
        
        if other_movements[i] > 0:
            points[2] += other_movements[i] - spy_movements[i]
        else:
            points[3] += (other_movements[i] - spy_movements[i])*(abs(other_movements[i] - spy_movements[i]))
    
    points_per_etf[ticker] = points
    print(ticker)

In [99]:
good_funds = []

for ticker in points_per_etf:
    points = points_per_etf[ticker]
    if points[0] >= 0 and points[0] + points[1] >= 0 and points[2] >= 0 and points[2] + points[3] >= 0:
        fund = pd.read_csv("Desktop/etf_historical_prices/{}.csv".format(ticker))
        if len(fund) < 756:
            continue
        good_funds.append(ticker)

In [ ]:
for fund in good_funds:
    print(fund, etf_data[fund]['name'], etf_data[fund]['morningstar_category'], etf_data[fund]['active_or_passive'])

In [ ]:
QTUM
FTEC
RYT
IGM
IXN
TDV
RGI
FIW
QQQ
QQQE
ONEQ
VONG
IWY
NULG

AVUS
XLK
VGT
IYW
QQEW
IWF

In [106]:
points_per_etf['IXN']

[210.80707427103303,
 -156.75595237001983,
 405.5905466775663,
 -305.1659680952048]